# Deploying TensorFlow Lite Models on Mobile and Edge Devices

1. Build a Model
2. Android Studio
3. Deployment 

In [8]:
import numpy as np
import tensorflow as tf

## Build a model

### Dataset

In [9]:
# Create a dataset with random input and output
x_train = np.random.rand(100, 1)
y_train = 2 * x_train + np.random.rand(100, 1) * 0.1

# Normalize the input
x_train_normalized = x_train / np.max(x_train)

# Save the dataset to a file
np.savez("dataset.npz", x_train=x_train_normalized, y_train=y_train)


### Load and save the  model

In [10]:
# Load the dataset
dataset = np.load("dataset.npz")
x_train = dataset["x_train"]
y_train = dataset["y_train"]

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))
])

# Compile and train the model
model.compile(optimizer="adam", loss="mse")
model.fit(x_train, y_train, epochs=1)

# Save the trained model
model.save("model.h5")


4/4 [==============================] - 1s 4ms/step - loss: 1.8550


### Convert model to Tensorflow lite

In [11]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\FIELDE~1\AppData\Local\Temp\tmpuk6208ly\assets


INFO:tensorflow:Assets written to: C:\Users\FIELDE~1\AppData\Local\Temp\tmpuk6208ly\assets


In [13]:
# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [14]:

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [15]:
input_details

[{'name': 'serving_default_dense_2_input:0',
  'index': 0,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 3,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [12]:
# Prepare input data
input_data = np.array([[0.5]], dtype=np.float32)

# Set the input tensor
interpreter.set_tensor(input_details[0]["index"], input_data)

# Run inference (i.e. make prediction using model)
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]["index"])
print("Output:", output_data)


Output: [[-0.15800251]]


### Android Studio
### Deployment 
1. Save tflite model to 'assets' folder
2. Add code 'implementation 'org.tensorflow:tensorflow-lite:2.7.0'' to 'build.gradle' file
3. Create Java Class
4. Setup device for deployment